In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, auc
from sklearn.utils import resample
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
import joblib
import shap

# **Modelo sin balance de clases**

In [ ]:
df = pd.read_csv('ARCHIVO.csv')

In [ ]:
# Revisar la distribución de la variable objetivo

print(df['DEFUNCION'].value_counts())

In [ ]:
# Variables que no ingresan al modelo

variables_no_utilizadas = ['FECHA_SINTOMAS', 'IDH_VALOR']
df = df.drop(columns=variables_no_utilizadas)

print(df.head())

In [ ]:
# Separar las características (X) de la variable objetivo (y)

X = df.drop('DEFUNCION', axis=1)
y = df['DEFUNCION']

# Dividir en conjuntos de entrenamiento y prueba con estratificación

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [ ]:
# Determinar el peso de las clases

ratio = y_train.value_counts()[0] / y_train.value_counts()[1]

# Configuración del modelo XGBoost

model = xgb.XGBClassifier(
    objective='binary:logistic',
    scale_pos_weight=5, # Se probó: ratio, 15, 8, 3
    eval_metric='auc', # Se probó: logloss, auc, error
    use_label_encoder=False
)

In [ ]:
# Entrenar el modelo

model.fit(X_train, y_train)

In [ ]:
# Hacer predicciones

y_pred = model.predict(X_test)

Evaluación

In [ ]:
# Imprimir el reporte de clasificación y la matriz de confusión

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

In [ ]:
# Predicción de probabilidades

y_pred_proba = model.predict_proba(X_test)[:, 1]

In [ ]:
# Calcular AUC-ROC

auc_roc = roc_auc_score(y_test, y_pred_proba)
print(f"AUC-ROC: {auc_roc:.4f}")

# Obtener las tasas de verdaderos positivos y falsos positivos para varios umbrales

fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)

# Graficar la curva ROC

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', label=f'AUC-ROC = {auc_roc:.4f}')
plt.plot([0, 1], [0, 1], color='red', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

# **Modelo con sobremuestreo**

In [ ]:
df = pd.read_csv('ARCHIVO.csv')

In [ ]:
# Revisar la distribución de la variable objetivo

print(df['DEFUNCION'].value_counts())

In [ ]:
# Variables que no ingresan al modelo

variables_no_utilizadas = ['FECHA_SINTOMAS', 'IDH_VALOR']
df = df.drop(columns=variables_no_utilizadas)

print(df.head())

In [ ]:
# Separar las características (X) de la variable objetivo (y)

X = df.drop('DEFUNCION', axis=1)
y = df['DEFUNCION']

In [ ]:
# Dividir en conjuntos de entrenamiento y prueba con estratificación

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [ ]:
# SMOTE para sobremuestrear DEFUNCION

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

In [ ]:
# Revisar la nueva distribución de clases después de SMOTE

print(f"Distribución después de SMOTE:\n{y_resampled.value_counts()}")

In [ ]:
# Configuración del modelo

model = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='auc',
    use_label_encoder=False,
    scale_pos_weight=1
)

In [ ]:
# Entrenar el modelo con el conjunto resampleado

model.fit(X_resampled, y_resampled)

In [ ]:
# Hacer predicciones

y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]

Evaluación

In [ ]:
# Imprimir el reporte de clasificación y la matriz de confusión

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

In [ ]:
# Calcular y graficar AUC-ROC

auc_roc = roc_auc_score(y_test, y_pred_proba)
print(f"AUC-ROC: {auc_roc:.4f}")

fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', label=f'AUC-ROC = {auc_roc:.4f}')
plt.plot([0, 1], [0, 1], color='red', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

# **Modelo con submuestreo**

In [ ]:
df = pd.read_csv('ARCHIVO.csv')

In [ ]:
# Revisar la distribución de la variable objetivo

print(df['DEFUNCION'].value_counts())

In [ ]:
# Variables que no ingresan al modelo

variables_no_utilizadas = ['FECHA_SINTOMAS', 'IDH_VALOR']
df = df.drop(columns=variables_no_utilizadas)

print(df.head())

In [ ]:
# Separar las características (X) de la variable objetivo (y)

X = df.drop('DEFUNCION', axis=1)
y = df['DEFUNCION']

In [ ]:
# Dividir en conjuntos de entrenamiento y prueba con estratificación

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [ ]:
# Aplicar submuestreo para balancear al 66%-33%

under_sampler = RandomUnderSampler(sampling_strategy=0.33, random_state=42)
X_under, y_under = under_sampler.fit_resample(X_train, y_train)

In [ ]:
# Revisar la nueva distribución de clases después del submuestreo

print(f"Distribución después del submuestreo:\n{y_under.value_counts()}")

In [ ]:
# Configuración del modelo XGBoost

model = xgb.XGBClassifier (
    objective='binary:logistic',
    learning_rate=0.01,
    max_depth=4,
    eval_metric='auc',
    min_child_weight=5,
    scale_pos_weight=2,
    gamma=1, # La regularización fue añadida por primera vez en el 7mo modelo
    subsample=0.8, # 7mo modelo
    colsample_bytree=0.8,
    use_label_encoder=False
)

In [ ]:
# Entrenar el modelo con parámetros manuales

model.fit(X_under, y_under)

In [ ]:
# Hacer predicciones (manual)

y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]

Evaluación

In [ ]:
# Imprimir el reporte de clasificación y la matriz de confusión

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

In [ ]:
# Calcular y graficar AUC-ROC

auc_roc = roc_auc_score(y_test, y_pred_proba)
print(f"AUC-ROC: {auc_roc:.4f}")

fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', label=f'AUC-ROC = {auc_roc:.4f}')
plt.plot([0, 1], [0, 1], color='red', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

**Información adicional**

En los modelos de sumbuestreo se intentó la búsqueda de hiperparámetros con grid search, éstas son las funciones:

In [ ]:
# Definir cuadrícula de hiperparámetros

 param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'min_child_weight': [1, 5, 10],
    'n_estimators': [100, 200, 300],
    'scale_pos_weight': [1, 3, 5]
}

In [ ]:
# Configurar la búsqueda exhaustiva

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='roc_auc', cv=3, verbose=2)

In [ ]:
# Aplicar la búsqueda con el conjunto resampleado

grid_search.fit(X_under, y_under)

In [ ]:
# Mejor combinación de hiperparámetros

print(f"Mejores parámetros: {grid_search.best_params_}")

In [ ]:
# Entrenar el modelo con parámetros encontrados

best_model = grid_search.best_estimator_
best_model.fit(X_under, y_under)

In [ ]:
# Hacer predicciones (Grid Search)

y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)[:, 1]

# **Modelo con divisiones previas (el mejor)**

In [ ]:
df = pd.read_csv('ARCHIVO.csv')

In [ ]:
# Revisar la distribución de la variable objetivo

print(df['DEFUNCION'].value_counts())

In [ ]:
# Variables que no ingresan al modelo

variables_no_utilizadas = ['FECHA_SINTOMAS', 'IDH_VALOR']
df = df.drop(columns=variables_no_utilizadas)

print(df.head())

In [ ]:
def balancear_datos(df, target_col='DEFUNCION'):
    # Filtrar las clases
    fallecidos = df[df[target_col] == 1]
    sobrevivientes = df[df[target_col] == 0]

    # Número de registros de fallecidos
    num_fallecidos = len(fallecidos)

    # Lista para almacenar las divisiones balanceadas
    divisiones = []

    # Generar divisiones con balance cercano al 50/50
    while len(sobrevivientes) >= num_fallecidos:
        # Muestrear aleatoriamente sobrevivientes sin reemplazo
        sobrevivientes_muestra = resample(sobrevivientes,
                                          replace=False,
                                          n_samples=num_fallecidos,
                                          random_state=42)

        # Crear un nuevo DataFrame balanceado
        df_balanceado = pd.concat([fallecidos, sobrevivientes_muestra])

        # Agregar la división a la lista
        divisiones.append(df_balanceado)

        # Eliminar la muestra seleccionada de los sobrevivientes originales
        sobrevivientes = sobrevivientes.drop(sobrevivientes_muestra.index)

    return divisiones

# Aplicar la función al DataFrame
divisiones_balanceadas = balancear_datos(df)

# divisiones_balanceadas ahora contiene los DataFrames balanceados

In [ ]:
# Resumen de divisiones creadas

for i, division in enumerate(divisiones_balanceadas):
    print(f"División {i + 1}: Fallecidos = {sum(division['DEFUNCION'] == 1)}, Sobrevivientes = {sum(division['DEFUNCION'] == 0)}")

Primera división

In [ ]:
random_seed = 196

# Crear el modelo base

model = xgb.XGBClassifier(objective='binary:logistic', random_state=random_seed)

In [ ]:
# Definir los rangos de parámetros a optimizar para cada segmento

# Segmento 1: max_depth y min_child_weight
param_grid_1 = {
    'max_depth': range(3, 16, 2),
    'min_child_weight': range(1, 7, 1)
}

# Segmento 2: gamma
param_grid_2 = {
    'gamma': [i / 10.0 for i in range(0, 6)]
}

# Segmento 3: subsample y colsample_bytree
param_grid_3 = {
    'subsample': [i / 10.0 for i in range(1, 11)],
    'colsample_bytree': [i / 10.0 for i in range(2, 11)]
}

# Segmento 4: reg_lambda y n_estimators
param_grid_4 = {
    'reg_lambda': [i / 10.0 for i in range(1, 11)],
    'n_estimators': range(50, 501, 50)
}

# Lista con los mejores parámetros
best_params = {}

Grid search para cada segmento de la primera división

In [ ]:
# Separar características y la variable objetivo

X = divisiones_balanceadas[0].drop('DEFUNCION', axis=1)
y = divisiones_balanceadas[0]['DEFUNCION']

# cross validation

cv = StratifiedKFold(n_splits=2, shuffle=True, random_state=random_seed)

for i, param_grid in enumerate([param_grid_1, param_grid_2, param_grid_3, param_grid_4], 1):
    print(f"Optimización del Segmento {i}: {list(param_grid.keys())}")
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='roc_auc',
                               cv=cv, verbose=1, n_jobs=-1)
    grid_search.fit(X, y)
    print(f"Mejores parámetros del Segmento {i}: {grid_search.best_params_}\n")

    model.set_params(**grid_search.best_params_)
    best_params.update(grid_search.best_params_)

print("Mejores parámetros finales:", best_params)

In [ ]:
# Partición entrenamiento-prueba con cross validation

train_test_splits = []

for i, division in enumerate(divisiones_balanceadas):
    X = division.drop('DEFUNCION', axis=1)
    y = division['DEFUNCION']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed, stratify=y)

    train_test_splits.append({
        'X_train': X_train,
        'X_test': X_test,
        'y_train': y_train,
        'y_test': y_test
    })

    print(f"División {i + 1}:")
    print(f"Tamaño del conjunto de entrenamiento: {X_train.shape[0]}")
    print(f"Tamaño del conjunto de prueba: {X_test.shape[0]}")
    print("\n")

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=random_seed)

In [ ]:
X_train = train_test_splits[0]['X_train']
y_train = train_test_splits[0]['y_train']
X_test = train_test_splits[0]['X_test']
y_test = train_test_splits[0]['y_test']

In [ ]:
# Entrenar con los mejores parámetros

model.fit(X_train, y_train)

# Mejores parámetros finales: {'max_depth': 3, 'min_child_weight': 5, 'gamma': 0.2,
# 'colsample_bytree': 1.0, 'subsample': 0.9, 'n_estimators': 100, 'reg_lambda': 0.9}

In [ ]:
# Hacer predicciones

y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]

Evaluación

In [ ]:
# Informe de clasificación

print("Classification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
# Matriz de confusión

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

In [ ]:
# Calcular y graficar AUC-ROC

fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'Curva ROC (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('Curva ROC')
plt.legend(loc='lower right')
plt.show()

# **Interpretación de modelos con SHAP**

In [ ]:
shap.initjs()

In [ ]:
explainer = shap.Explainer(model)

In [ ]:
shap_values = explainer(X)

In [ ]:
# Gráfico de resumen (summary plot)

shap.summary_plot(shap_values, X)

In [ ]:
# Gráfico de dependencia (Dependence Plot)

shap.plots.scatter(shap_values[:, 'NEUMONIA'])

In [ ]:
# Gráfico de fuerza (Force Plot)

# Para una ocurrencia específica
shap.force_plot(explainer.expected_value, shap_values[0].values, X.iloc[0,:])

In [ ]:
# Gráfico de valores SHAP (SHAP Values Plot)

# Para la primera ocurrencia
shap.waterfall_plot(shap_values[0])